# 第17章潜在语义分析

## 习题17.1
&emsp;&emsp;试将图17.1的例子进行潜在语义分析，并对结果进行观察。

**解答：**  

**解答思路：**

**解答步骤：**  

## 习题17.2
&emsp;&emsp;给出损失函数是散度损失时的非负矩阵分解（潜在语义分析）的算法。

**解答：**

**解答思路：**  

**解答步骤：**   

## 习题17.3
&emsp;&emsp;给出潜在语义分析的两种算法的计算复杂度，包括奇异值分解法和非负矩阵分解法。

**解答：**

**解答思路：**  

**解答步骤：**   

## 习题17.4
&emsp;&emsp;列出潜在语义分析与主成分分析的异同。

**解答：**

**解答思路：**  

**解答步骤：**   